In [145]:
d = {'a': {'a':1,'t':-1,'c':-1,'g':-1}, 
     't': {'a':-1,'t':1,'c':-1,'g':-1}, 
     'c': {'a':-1,'t':-1,'c':1,'g':-1}, 
     'g': {'a':-1,'t':-1,'c':-1,'g':1}}

## Substitution matrices

Given the problem statement, that transversions between, A<->G & T<->C are less common than, A<->T, A<->C, G<->T & G<->C.
We can use the following substitution matrix instead:

     A  G  T  C
________________
A |  1 -2 -1 -1
G | -2  1 -1 -1
T | -1 -1  1 -2
C | -1 -1 -2  1

Note, that the lower probability transversions were given a lower score to account for them being less frequent.

## Global Alignment

In [146]:
class Solution:
    def traceback(self, directions, matrix, i, j, path_traced, is_local=False):
        current_element = matrix[i][j]
        print('Current traceback: ' + str(path_traced))
        
        if i == j and i == 0 and not is_local:
            path_traced.append((0, 0))
            return path_traced
        
        # assumes there's only one valid direction??
        for direction in directions[i][j]:
            print('Current direction: ' + str(direction))
            path_traced.append((i, j))
            print('Current path_traced: ' + str(path_traced))
            if direction == 'diag':
                return self.traceback(directions, matrix, i - 1, j - 1, path_traced)
            if direction == 'left':
                return self.traceback(directions, matrix, i, j - 1, path_traced)
            elif direction == 'top':
                return self.traceback(directions, matrix, i - 1, j, path_traced)
        
        return path_traced
    
    def find_max_index_and_val(self, matrix):
        # returns a tuple with i, j & max_val
        current_max = -999999  # arbitraty value
        max_i, max_j = -1, -1
        for i in range(len(matrix)):
            for j in range(len(matrix[0])):
                if matrix[i][j] > current_max:
                    max_i, max_j = i, j
                    current_max = matrix[i][j]
                    
        return max_i, max_j, current_max
    
    def is_matrix_zeros(self, matrix):
        current_sum = 0
        for row in matrix:
            current_sum += sum(row)
        
        if current_sum == 0:
            return True
        return False
    
    def set_max_path_to_zeros(self, matrix, path_traced):
        for entry in path_traced:
            print('Setting entry to 0: ' + str(entry))
            matrix[entry[0]][entry[1]] = 0
    
    def local_traceback_helper(self, directions, matrix):
        all_paths = []
        i, j, max_val = self.find_max_index_and_val(matrix)
        while (not self.is_matrix_zeros(matrix)):
            print('Helper tracing path for max_val: ' + str(max_val))
            path_traced = self.traceback(directions, matrix, i, j, [])
            self.set_max_path_to_zeros(matrix, path_traced)
            all_paths.append(path_traced)
            i, j, max_val = self.find_max_index_and_val(matrix)
            
            
        return all_paths
    
    def fill_gap_values(self, matrix, num_rows, num_cols, gap):
        # fill the first row & first column with the gap values
        curr_gap = 0
        for j in range(num_cols):
            matrix[0][j] = curr_gap
            curr_gap += gap
            
        curr_gap = 0
        for i in range(num_rows):
            matrix[i][0] = curr_gap
            curr_gap += gap
            
    def calc_directions(self, matrix, gap, num_rows, num_cols, sequence_A, sequence_B, substitution, is_local=False):
        directions = [[[] for i in range(num_cols)] for j in range(num_rows)]
        # now navigate the matrix & perform the global alignment
        for i in range(1, num_rows):
            for j in range(1, num_cols):
                left_val = matrix[i][j - 1] + gap
                top_val = matrix[i - 1][j] + gap
                diag_val = matrix[i - 1][j - 1] + substitution[sequence_A[i - 1]][sequence_B[j - 1]]
                if (is_local):
                    left_val, top_val, diag_val = max(left_val, 0), max(top_val, 0), max(diag_val, 0)
                max_val = max(left_val, top_val, diag_val)
                matrix[i][j] = max_val
                
                # check if this simplification works
                if (is_local and max_val == 0):
                    continue
                
                if (left_val == max_val):
                    directions[i][j].append('left')
                if (top_val == max_val):
                    directions[i][j].append('top')
                if (diag_val == max_val):
                    directions[i][j].append('diag')
                    
        return directions
    
    def calc_alignment(self, path_traced, sequence_A, sequence_B, is_local=False):
        alignment_seq_a = ''
        alignment_seq_b = ''
        first_time_local = True
        
        # TODO: error handling
        for i in range(1, len(path_traced)):
            previous_path = path_traced[i-1]
            current_path = path_traced[i]
            if (is_local and first_time_local):
                alignment_seq_a += sequence_A[previous_path[0] - 1]
                alignment_seq_b += sequence_B[previous_path[1] - 1]
                first_time_local = False
            
            print('Previous path: ' + str(previous_path))
            print('Current path: ' + str(current_path))
            # current_path direction was top
            if (current_path[0] - previous_path[0] == 1 and current_path[1] == previous_path[1]):
                alignment_seq_a += sequence_A[current_path[0] - 1]
                alignment_seq_b += '-'
                
            elif (current_path[1] - previous_path[1] == 1 and current_path[0] == previous_path[0]):
                alignment_seq_a += '-'
                alignment_seq_b += sequence_B[current_path[1] - 1]
                
            elif (current_path[0] - previous_path[0] == 1 and current_path[1] - previous_path[1] == 1):
                alignment_seq_a += sequence_A[current_path[0] - 1]
                alignment_seq_b += sequence_B[current_path[1] - 1]
             
        if (alignment_seq_a == alignment_seq_b and alignment_seq_a == ''):
            return []
        return [(alignment_seq_a, alignment_seq_b)]
    
    
    def global_alignment(self, sequence_A: str, sequence_B: str, substitution: dict, gap: int) -> [tuple]:
        num_cols = len(sequence_B) + 1
        num_rows = len(sequence_A) + 1
        matrix = [[0 for i in range(num_cols)] for j in range(num_rows)]
        
        self.fill_gap_values(matrix, num_rows, num_cols, gap)
        directions = self.calc_directions(matrix, gap, num_rows, num_cols, sequence_A, sequence_B, substitution)
        print('Now matrix is: ')
        for row in matrix:
            print(row)
            
        print('Directions are :')
        for row in directions:
            print(row)        # now traceback, by starting the bottom right of the matrix
        path_traced = self.traceback(directions, matrix, num_rows - 1, num_cols - 1, [])
        path_traced = path_traced[-1::-1]
        # for traceback path, calculate the alignment
        return self.calc_alignment(path_traced, sequence_A, sequence_B)

                         
    def local_alignment(self, sequence_A: str, sequence_B:str, substitution: dict, gap: int) -> [tuple]:
        num_cols = len(sequence_B) + 1
        num_rows = len(sequence_A) + 1
        matrix = [[0 for i in range(num_cols)] for j in range(num_rows)]
        
        directions = self.calc_directions(matrix, gap, num_rows, num_cols, sequence_A, sequence_B, substitution, is_local=True)
        print('Now matrix is: ')
        for row in matrix:
            print(row)
            
        print('Directions are :')
        for row in directions:
            print(row)
            
        paths_traced = self.local_traceback_helper(directions, matrix)
        all_alignments = []
        for path_list in paths_traced:
            calculated_alignment = self.calc_alignment(path_list[-1::-1], sequence_A, sequence_B, True)
            if calculated_alignment != []:
                for entry in calculated_alignment:
                    all_alignments.append(entry)
                    
        print('Paths are:')
        print(paths_traced)
        
        return all_alignments

In [150]:
s = Solution()

# TODO: add lowercasing support - so that it works even if the casing is messed up in the input
d = {'a': {'a':1,'t':-1,'c':-1,'g':-1}, 
     't': {'a':-1,'t':1,'c':-1,'g':-1}, 
     'c': {'a':-1,'t':-1,'c':1,'g':-1}, 
     'g': {'a':-1,'t':-1,'c':-1,'g':1}}

s.global_alignment('gata', 'ctac', d, -2)

Now matrix is: 
[0, -2, -4, -6, -8]
[-2, -1, -3, -5, -7]
[-4, -3, -2, -2, -4]
[-6, -5, -2, -3, -3]
[-8, -7, -4, -1, -3]
Directions are :
[[], [], [], [], []]
[[], ['diag'], ['left', 'diag'], ['left', 'diag'], ['left', 'diag']]
[[], ['top', 'diag'], ['diag'], ['diag'], ['left']]
[[], ['top', 'diag'], ['diag'], ['diag'], ['diag']]
[[], ['top', 'diag'], ['top'], ['diag'], ['left']]
Current traceback: []
Current direction: left
Current path_traced: [(4, 4)]
Current traceback: [(4, 4)]
Current direction: diag
Current path_traced: [(4, 4), (4, 3)]
Current traceback: [(4, 4), (4, 3)]
Current direction: diag
Current path_traced: [(4, 4), (4, 3), (3, 2)]
Current traceback: [(4, 4), (4, 3), (3, 2)]
Current direction: top
Current path_traced: [(4, 4), (4, 3), (3, 2), (2, 1)]
Current traceback: [(4, 4), (4, 3), (3, 2), (2, 1)]
Current direction: diag
Current path_traced: [(4, 4), (4, 3), (3, 2), (2, 1), (1, 1)]
Current traceback: [(4, 4), (4, 3), (3, 2), (2, 1), (1, 1)]
Previous path: (0, 0)
Curre

[('gata-', 'c-tac')]

In [148]:
s = Solution()

# TODO: add lowercasing support - so that it works even if the casing is messed up in the input
d = {'a': {'a':1,'t':-1,'c':-1,'g':-1}, 
     't': {'a':-1,'t':1,'c':-1,'g':-1}, 
     'c': {'a':-1,'t':-1,'c':1,'g':-1}, 
     'g': {'a':-1,'t':-1,'c':-1,'g':1}}

# s.global_alignment('gata', 'ctac', d, -2)
s.local_alignment('gata', 'ctac', d, -2)
# gata, -cta ???

Now matrix is: 
[0, 0, 0, 0, 0]
[0, 0, 0, 0, 0]
[0, 0, 0, 1, 0]
[0, 0, 1, 0, 0]
[0, 0, 0, 2, 0]
Directions are :
[[], [], [], [], []]
[[], [], [], [], []]
[[], [], [], ['diag'], []]
[[], [], ['diag'], [], []]
[[], [], [], ['diag'], []]
Helper tracing path for max_val: 2
Current traceback: []
Current direction: diag
Current path_traced: [(4, 3)]
Current traceback: [(4, 3)]
Current direction: diag
Current path_traced: [(4, 3), (3, 2)]
Current traceback: [(4, 3), (3, 2)]
Setting entry to 0: (4, 3)
Setting entry to 0: (3, 2)
Helper tracing path for max_val: 1
Current traceback: []
Current direction: diag
Current path_traced: [(2, 3)]
Current traceback: [(2, 3)]
Setting entry to 0: (2, 3)
Previous path: (3, 2)
Current path: (4, 3)
Paths are:
[[(4, 3), (3, 2)], [(2, 3)]]


[('ta', 'ta')]

In [149]:
## TODO: Put this in a separate file

## TODO: pretty print logic

def custom_alignment(firstname, lastname, compare_str, gap):
    name = firstname + lastname
    uniq_letters = set(name)
    english_alpha = 'abcdefghijklmnopqrstuvwxyz'
    english_alpha_set = set(english_alpha)
    substitution_dict = {}
    match_factor = 2
    semi_match_factor = 1
    mismatch_factor = -1
    for letter in english_alpha:
        substitution_dict[letter] = {}
        for letter2 in english_alpha:
            if (letter == letter2):
                substitution_dict[letter][letter2] = match_factor
            elif (letter in uniq_letters and letter2 in uniq_letters):  # semi matches
                substitution_dict[letter][letter2] = semi_match_factor
            else:
                substitution_dict[letter][letter2] = mismatch_factor
                
#     print('Custom subs. dict: ' + str(substitution_dict))
    return s.local_alignment(name, compare_str, substitution_dict, gap)
    
firstname = 'devi'
lastname = 'kandimalla'
compare_str = 'thequickbrownfoxjumpsoverthelazydog'
gap = -2

# TODO: verify a couple of the custom alignments
custom_alignment(firstname, lastname, compare_str, gap)

Now matrix is: 
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 2, 0, 0]
[0, 0, 0, 2, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 3, 1, 0, 0, 2, 2, 2, 0, 0, 1, 1, 0]
[0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 2, 2, 2, 0, 0, 1, 3, 3, 1, 0, 1, 0, 0]
[0, 0, 0, 1, 0, 0, 2, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 3, 1, 1, 0, 1, 2, 4, 2, 0, 1, 0, 0]
[0, 0, 0, 1, 0, 0, 1, 1, 2, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 2, 2, 0, 0, 1, 2, 3, 3, 1, 1, 0, 0]
[0, 0, 0, 1, 0, 0, 1, 0, 2, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 2, 1, 1, 0, 1, 2, 4, 2, 2, 2, 0, 0]
[0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 2, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 2, 1, 0, 0, 1, 2, 3, 3, 1, 3, 1, 0]
[0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 2, 1, 0, 0, 1, 2, 3, 2, 2, 3, 2, 0]
[0,

[('lla', 'ela'),
 ('evi', 'ela'),
 ('ika', 'ela'),
 ('ima', 'ela'),
 ('all', 'ela'),
 ('de', 've'),
 ('dev', 'ela'),
 ('vi', 've'),
 ('vik', 'ela'),
 ('evik', 'elaz'),
 ('kan', 'ela'),
 ('ikan', 'elaz'),
 ('evikan', 'elazyd'),
 ('and', 'ela'),
 ('ikan-d', 'elazyd'),
 ('ndi', 'ela'),
 ('ikandi', 'elazyd'),
 ('dim', 'ela'),
 ('mal', 'ela'),
 ('imal', 'elaz'),
 ('lla-', 'elaz'),
 ('imalla', 'elazyd'),
 ('de', 'la'),
 ('ev', 've'),
 ('dev', 'ver'),
 ('evi-', 'elaz'),
 ('ik', 've'),
 ('vik', 'ver'),
 ('ika', 'ick'),
 ('ka', 've'),
 ('ika-', 'elaz'),
 ('evik-a', 'elazyd'),
 ('an', 've'),
 ('nd', 've'),
 ('kand', 'elaz'),
 ('evikand', 'elazydo'),
 ('di', 've'),
 ('andi', 'elaz'),
 ('ikan-di', 'elazydo'),
 ('im', 've'),
 ('ndim', 'elaz'),
 ('kandim', 'elazyd'),
 ('ikandim', 'elazydo'),
 ('ima', 'ick'),
 ('ma', 've'),
 ('ima-', 'elaz'),
 ('andima', 'elazyd'),
 ('al', 've'),
 ('ndimal', 'elazyd'),
 ('ll', 've'),
 ('all-', 'elaz'),
 ('imal-l', 'elazyd'),
 ('la', 've'),
 ('la', 'el'),
 ('de-', 've